### Sử dụng DASK để đọc file Sản lượng có nhiều Sheets

In [16]:
# -*- coding: utf-8 -*-
import pandas as pd
import os
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output # Import clear_output
import calendar

print("--- Bắt đầu Script ---")

# ========== 1. ĐỌC VÀ GHÉP DỮ LIỆU PARQUET ==========
print(">>> Bước 1: Đọc và ghép dữ liệu Parquet...")
try:
    # --- Cấu hình ---
    # !!! THAY ĐỔI ĐƯỜNG DẪN NÀY CHO PHÙ HỢP VỚI MÁY CỦA BẠN !!!
    parquet_folder = r"C:\Khue\TDN\data\processed"
    
    if not os.path.isdir(parquet_folder):
        print(f"❌ LỖI: Thư mục không tồn tại: {parquet_folder}")
        exit() # Thoát nếu thư mục không đúng

    # --- Tìm file ---
    parquet_files = [os.path.join(parquet_folder, f)
                     for f in os.listdir(parquet_folder)
                     if f.startswith("sanluong_") and f.endswith(".parquet")]

    if not parquet_files:
        print(f"❌ LỖI: Không tìm thấy file nào theo mẫu 'sanluong_*.parquet' trong thư mục: {parquet_folder}")
        exit() # Thoát nếu không có file

    # --- Đọc và ghép ---
    df_list = []
    for f in parquet_files:
        try:
            df_list.append(pd.read_parquet(f))
        except Exception as e:
            print(f"⚠️ Cảnh báo: Không thể đọc file {f}. Lỗi: {e}")
    
    if not df_list:
        print(f"❌ LỖI: Không đọc được file parquet nào thành công.")
        exit()

    df_all = pd.concat(df_list, ignore_index=True)

    print(f"✅ Đã đọc xong {len(df_list)}/{len(parquet_files)} file parquet.") # Hiển thị số file đọc thành công
    print(f"👉 Tổng số dòng dữ liệu ban đầu: {df_all.shape[0]:,}")

except Exception as e:
    print(f"❌ LỖI nghiêm trọng trong quá trình đọc dữ liệu: {e}")
    exit()

# ========== 2. CHỌN LỌC VÀ CHUẨN BỊ DỮ LIỆU ==========
print("\n>>> Bước 2: Chọn lọc và chuẩn bị dữ liệu...")
try:
    # --- Chọn cột và đổi tên ---
    required_cols = ['CTDL', 'NMTD', 'MADIEMDO', 'ENDTIME', 'CS']
    if not all(col in df_all.columns for col in required_cols):
        print(f"❌ LỖI: Thiếu cột cần thiết trong dữ liệu. Cần có: {required_cols}")
        missing_cols = [col for col in required_cols if col not in df_all.columns]
        print(f"   Các cột bị thiếu: {missing_cols}")
        exit()

    df_sanluong = df_all[required_cols].copy() # Sử dụng .copy() để tránh SettingWithCopyWarning
    df_sanluong.rename(columns={'ENDTIME': 'TIME'}, inplace=True)

    # --- Chuyển đổi cột TIME sang datetime và xử lý lỗi ---
    df_sanluong['TIME'] = pd.to_datetime(df_sanluong['TIME'], errors='coerce')
    original_rows = len(df_sanluong)
    df_sanluong.dropna(subset=['TIME'], inplace=True) # Loại bỏ các dòng có TIME không hợp lệ
    if len(df_sanluong) < original_rows:
        print(f"⚠️ Cảnh báo: Đã loại bỏ {original_rows - len(df_sanluong)} dòng do giá trị TIME không hợp lệ.")

    if df_sanluong.empty:
        print("❌ LỖI: Không còn dữ liệu sau khi xử lý cột TIME.")
        exit()
        
    # --- Tạo cột thời gian ---
    df_sanluong['YEAR'] = df_sanluong['TIME'].dt.year
    df_sanluong['MONTH_NUM'] = df_sanluong['TIME'].dt.month
    df_sanluong['DAY_NUM'] = df_sanluong['TIME'].dt.day

    print("✅ Dữ liệu đã được chọn lọc và chuẩn bị.")
    print(f"👉 Số dòng dữ liệu sau khi chuẩn bị: {df_sanluong.shape[0]:,}")

except Exception as e:
    print(f"❌ LỖI trong quá trình chuẩn bị dữ liệu: {e}")
    exit()

# ========== 3. TẠO GIAO DIỆN TƯƠNG TÁC ==========
print("\n>>> Bước 3: Tạo giao diện tương tác...")

# --- Dữ liệu cho Dropdowns ---
try:
    ctdl_list = sorted(df_sanluong['CTDL'].dropna().unique())
    year_list = sorted(df_sanluong['YEAR'].dropna().unique())
    
    if not ctdl_list: print("⚠️ Cảnh báo: Không tìm thấy giá trị CTDL nào.")
    if not year_list: print("⚠️ Cảnh báo: Không tìm thấy giá trị YEAR nào.")
        
except Exception as e:
    print(f"❌ LỖI khi lấy dữ liệu cho dropdown: {e}")
    exit()

# --- Hàm tạo Dropdown ---
def make_dropdown(options, description, width='250px', margin='0px 20px 0px 0px'):
    # Đảm bảo options luôn là list, ngay cả khi rỗng
    options_list = list(options) if options is not None else []
    # Chọn giá trị mặc định nếu có
    default_value = options_list[0] if options_list else None
    return widgets.Dropdown(
        options=options_list,
        value=default_value, # Đặt giá trị mặc định
        description=description,
        layout=widgets.Layout(width=width, margin=margin),
        style={'description_width': 'auto'},
        disabled=not options_list # Vô hiệu hóa nếu không có lựa chọn
    )

# --- Tạo Widgets ---
ctdl_dropdown_month = make_dropdown(ctdl_list, 'CTDL:', '300px')
nmtd_dropdown_month = make_dropdown([], 'Nhà máy:', '300px') # Khởi tạo rỗng
year_dropdown_month = make_dropdown(year_list, 'Năm:', '120px')
month_dropdown_month = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
day_placeholder = widgets.Label(value='', layout=widgets.Layout(width='120px', margin='0px 20px 0px 0px'))

ctdl_dropdown_day = make_dropdown(ctdl_list, 'CTDL:', '300px')
nmtd_dropdown_day = make_dropdown([], 'Nhà máy:', '300px') # Khởi tạo rỗng
year_dropdown_day = make_dropdown(year_list, 'Năm:', '120px')
month_dropdown_day = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
day_dropdown_day = make_dropdown([], 'Ngày:', '120px') # Khởi tạo rỗng, sẽ được cập nhật

# --- Nhóm Widgets ---
month_widgets = [ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month]
day_widgets = [ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day]

# --- Output Widget ---
out = widgets.Output()

# ========== 4. LOGIC TƯƠNG TÁC VÀ VẼ BIỂU ĐỒ ==========
print(">>> Bước 4: Thiết lập logic tương tác và vẽ biểu đồ...")

# --- Cập nhật NMTD theo CTDL ---
def update_nmtd(ctdl_value, dropdown_to_update):
    # Tạm thời tắt observe của dropdown cần cập nhật
    dropdown_to_update.unobserve_all()
    
    if ctdl_value:
        filtered = df_sanluong[df_sanluong['CTDL'] == ctdl_value]
        options = sorted(filtered['NMTD'].dropna().unique())
        dropdown_to_update.options = options
        dropdown_to_update.value = options[0] if options else None # Đặt giá trị đầu tiên nếu có
        dropdown_to_update.disabled = not options # Vô hiệu hóa nếu không có lựa chọn
    else:
        dropdown_to_update.options = []
        dropdown_to_update.value = None
        dropdown_to_update.disabled = True
        
    # Bật lại observe (nếu cần - nhưng ở đây logic vẽ lại được trigger bởi CTDL nên không cần bật lại ngay)

# --- Giới hạn Ngày theo Tháng/Năm ---
def update_day_options(*args):
    # Tạm thời tắt observe của day_dropdown_day
    day_dropdown_day.unobserve(on_change_day, names='value')
    
    year = year_dropdown_day.value
    month = month_dropdown_day.value
    
    if year and month:
        try:
            max_day = calendar.monthrange(year, month)[1]
            day_options = list(range(1, max_day + 1))
            current_day_value = day_dropdown_day.value
            
            day_dropdown_day.options = day_options
            # Giữ lại giá trị ngày nếu còn hợp lệ, nếu không thì chọn ngày 1
            if current_day_value in day_options:
                day_dropdown_day.value = current_day_value
            else:
                 day_dropdown_day.value = day_options[0] if day_options else None
            day_dropdown_day.disabled = not day_options
        except ValueError: # Xử lý trường hợp tháng/năm không hợp lệ (dù ít xảy ra với dropdown)
             day_dropdown_day.options = []
             day_dropdown_day.value = None
             day_dropdown_day.disabled = True
    else:
        day_dropdown_day.options = []
        day_dropdown_day.value = None
        day_dropdown_day.disabled = True
        
    # Bật lại observe
    day_dropdown_day.observe(on_change_day, names='value')


# --- Hàm Vẽ Biểu đồ ---
def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
    with out: # Hiển thị trong widget Output 'out'
        clear_output(wait=True) # Xóa output cũ trước khi vẽ mới

        # Kiểm tra giá trị đầu vào cơ bản
        if not all([ctdl, nmtd, year, month]):
             print("Vui lòng chọn đầy đủ CTDL, Nhà máy, Năm và Tháng.")
             return
        if mode == 'day' and not day:
             print("Vui lòng chọn Ngày.")
             return

        # Lọc dữ liệu
        try:
            base_filter = (
                (df_sanluong['CTDL'] == ctdl) &
                (df_sanluong['NMTD'] == nmtd) &
                (df_sanluong['YEAR'] == year) &
                (df_sanluong['MONTH_NUM'] == month)
            )
            if mode == 'month':
                filtered = df_sanluong[base_filter]
                title_ext = f"{month}/{year}"
            else: # mode == 'day'
                filtered = df_sanluong[base_filter & (df_sanluong['DAY_NUM'] == day)]
                title_ext = f"{day}/{month}/{year}"

            if filtered.empty:
                print(f"Không có dữ liệu cho lựa chọn: {ctdl} - {nmtd} - {title_ext}")
                return

            filtered = filtered.sort_values('TIME')

            # Vẽ biểu đồ
            fig = px.line(
                filtered,
                x='TIME',
                y='CS',
                color='MADIEMDO',
                render_mode='webgl',
                markers=True if mode == 'day' else False,
                labels={'CS': 'Công suất', 'TIME': 'Thời gian', 'MADIEMDO': 'Điểm đo'} # Nhãn rõ ràng hơn
            )

            fig.update_layout(
                title=dict( # Cách đặt tiêu đề khác, căn giữa tốt hơn
                   text=f"<b>Công suất theo chu kỳ - {nmtd} - {title_ext}</b>",
                   x=0.5, # Căn giữa
                   xanchor='center',
                   font=dict(size=18)
                ),
                template='plotly_white',
                height=600,
                hovermode='x unified',
                xaxis_title="Thời gian", # Đặt tiêu đề trực tiếp
                yaxis_title='Giá trị công suất',
                xaxis=dict(showgrid=True, gridcolor='lightgrey', griddash='dot'),
                yaxis=dict(showgrid=True, gridcolor='lightgrey', griddash='dot', rangemode='tozero'),
                margin=dict(l=60, r=40, t=80, b=80), # Tăng bottom margin nếu tiêu đề trục X bị cắt
                legend_title_text='Điểm đo' # Tiêu đề cho legend
            )

            fig.update_traces(
                line=dict(width=2),
                hovertemplate='Thời gian: %{x|%Y-%m-%d %H:%M}<br>Công suất: %{y:.2f}<br>Điểm đo: %{fullData.name}' # Định dạng hover
            )

            fig.show() # Hiển thị biểu đồ

        except Exception as e:
            print(f"❌ LỖI khi lọc hoặc vẽ biểu đồ: {e}")


# --- Các hàm xử lý sự kiện thay đổi ---
def on_change_month(change):
    if change['new'] is not None: # Chỉ chạy nếu có giá trị mới
       plot_filtered(
           'month',
           ctdl_dropdown_month.value,
           nmtd_dropdown_month.value,
           year_dropdown_month.value,
           month_dropdown_month.value
       )

def on_change_day(change):
     if change['new'] is not None: # Chỉ chạy nếu có giá trị mới
        plot_filtered(
            'day',
            ctdl_dropdown_day.value,
            nmtd_dropdown_day.value,
            year_dropdown_day.value,
            month_dropdown_day.value,
            day_dropdown_day.value
        )

# --- Hàm bật/tắt Observe ---
def set_observe(widgets_list, callback, enable=True):
    for w in widgets_list:
        if enable:
            w.observe(callback, names='value')
        else:
            w.unobserve(callback, names='value')

# --- Gắn các sự kiện Observe ---

# Sự kiện cập nhật NMTD khi CTDL thay đổi
ctdl_dropdown_month.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_month), names='value')
ctdl_dropdown_day.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_day), names='value')

# Sự kiện cập nhật Ngày khi Năm/Tháng thay đổi (chỉ cho tab Ngày)
year_dropdown_day.observe(update_day_options, names='value')
month_dropdown_day.observe(update_day_options, names='value')

# Sự kiện vẽ lại biểu đồ khi có thay đổi ở các dropdown
set_observe(month_widgets, on_change_month, True)
set_observe(day_widgets, on_change_day, True)


# --- Khởi tạo trạng thái ban đầu ---
print(">>> Khởi tạo trạng thái ban đầu...")
update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)
update_day_options() # Cập nhật ngày cho tab Ngày

# ========== 5. TẠO LAYOUT VÀ HIỂN THỊ ==========
print(">>> Bước 5: Tạo layout và hiển thị giao diện (Cần môi trường hỗ trợ ipywidgets)...")

# --- Layouts cho Tabs ---
tab_thang = widgets.HBox([
    ctdl_dropdown_month,
    nmtd_dropdown_month,
    year_dropdown_month,
    month_dropdown_month,
    day_placeholder
], layout=widgets.Layout(flex_flow='row wrap', justify_content='flex-start')) # Cho phép xuống dòng nếu không đủ chỗ

tab_ngay = widgets.HBox([
    ctdl_dropdown_day,
    nmtd_dropdown_day,
    year_dropdown_day,
    month_dropdown_day,
    day_dropdown_day
], layout=widgets.Layout(flex_flow='row wrap', justify_content='flex-start'))

# --- Tạo Tabs ---
tabs = widgets.Tab(children=[tab_thang, tab_ngay])
tabs.set_title(0, 'Xem theo tháng')
tabs.set_title(1, 'Xem theo ngày')

# --- Logic chuyển Tab (Đồng bộ hóa giá trị) ---
def on_tab_change(change):
    if change['name'] == 'selected_index':
        new_index = change['new']
        
        # Tắt tất cả observe vẽ lại để tránh trigger khi đang gán giá trị
        set_observe(month_widgets, on_change_month, False)
        set_observe(day_widgets, on_change_day, False)
        # Tạm thời tắt observe cập nhật ngày
        year_dropdown_day.unobserve(update_day_options, names='value')
        month_dropdown_day.unobserve(update_day_options, names='value')
        
        try: # Bọc trong try...finally để đảm bảo observe được bật lại
            if new_index == 1: # Chuyển sang tab Ngày
                # Đồng bộ giá trị từ Tháng sang Ngày
                ctdl_dropdown_day.value = ctdl_dropdown_month.value
                # update_nmtd sẽ được trigger bởi observe của ctdl_dropdown_day, không cần gán trực tiếp
                # Chờ NMTD cập nhật rồi mới gán (hoặc gọi update_nmtd thủ công nếu cần ngay)
                update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day) # Gọi thủ công để đảm bảo NMTD có giá trị
                year_dropdown_day.value = year_dropdown_month.value
                month_dropdown_day.value = month_dropdown_month.value
                update_day_options() # Cập nhật lại ngày sau khi có tháng/năm mới
                # Chọn ngày đầu tiên nếu có thể
                if day_dropdown_day.options:
                   day_dropdown_day.value = day_dropdown_day.options[0]
                   
                # Gọi vẽ lại cho tab Ngày
                # Cần đảm bảo các giá trị đã ổn định trước khi vẽ
                if all([ctdl_dropdown_day.value, nmtd_dropdown_day.value, year_dropdown_day.value, month_dropdown_day.value, day_dropdown_day.value]):
                   plot_filtered('day', ctdl_dropdown_day.value, nmtd_dropdown_day.value, year_dropdown_day.value, month_dropdown_day.value, day_dropdown_day.value)
                else:
                    with out:
                       clear_output(wait=True)
                       print("Vui lòng chọn đầy đủ thông tin cho tab Ngày.")


            elif new_index == 0: # Chuyển sang tab Tháng
                # Đồng bộ giá trị từ Ngày sang Tháng
                ctdl_dropdown_month.value = ctdl_dropdown_day.value
                update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month) # Gọi thủ công
                year_dropdown_month.value = year_dropdown_day.value
                month_dropdown_month.value = month_dropdown_day.value
                
                # Gọi vẽ lại cho tab Tháng
                if all([ctdl_dropdown_month.value, nmtd_dropdown_month.value, year_dropdown_month.value, month_dropdown_month.value]):
                    plot_filtered('month', ctdl_dropdown_month.value, nmtd_dropdown_month.value, year_dropdown_month.value, month_dropdown_month.value)
                else:
                    with out:
                       clear_output(wait=True)
                       print("Vui lòng chọn đầy đủ thông tin cho tab Tháng.")

        finally:
             # Bật lại observe cập nhật ngày
             year_dropdown_day.observe(update_day_options, names='value')
             month_dropdown_day.observe(update_day_options, names='value')
             # Bật lại observe vẽ lại cho cả hai tab
             set_observe(month_widgets, on_change_month, True)
             set_observe(day_widgets, on_change_day, True)


tabs.observe(on_tab_change, names='selected_index')

# --- Hiển thị Giao diện ---
# Điều này chỉ hoạt động trong môi trường hỗ trợ IPython display (Jupyter, VSCode-Jupyter, Colab, Voilà...)
if __name__ == "__main__":
    print("--- Hiển thị giao diện ---")
    # Trigger vẽ biểu đồ ban đầu cho tab mặc định (Tháng)
    if all([ctdl_dropdown_month.value, nmtd_dropdown_month.value, year_dropdown_month.value, month_dropdown_month.value]):
         plot_filtered(
             'month',
             ctdl_dropdown_month.value,
             nmtd_dropdown_month.value,
             year_dropdown_month.value,
             month_dropdown_month.value
         )
    else:
        with out:
            clear_output(wait=True)
            print("Khởi tạo hoàn tất. Vui lòng chọn các bộ lọc để xem dữ liệu.")
            
    display(tabs, out)
    print("--- Script hoàn thành (Giao diện đã được hiển thị nếu môi trường hỗ trợ) ---")

--- Bắt đầu Script ---
>>> Bước 1: Đọc và ghép dữ liệu Parquet...
✅ Đã đọc xong 4/4 file parquet.
👉 Tổng số dòng dữ liệu ban đầu: 18,404,864

>>> Bước 2: Chọn lọc và chuẩn bị dữ liệu...
✅ Dữ liệu đã được chọn lọc và chuẩn bị.
👉 Số dòng dữ liệu sau khi chuẩn bị: 18,404,864

>>> Bước 3: Tạo giao diện tương tác...
>>> Bước 4: Thiết lập logic tương tác và vẽ biểu đồ...
>>> Khởi tạo trạng thái ban đầu...


TraitError: Invalid selection: value not found

In [1]:
# import dask.dataframe as dd
# import pandas as pd
# import warnings
# warnings.filterwarnings('ignore')

# # Các chị thay đường dẫn file sản lượng vào phần này ạ
# file_path = r"C:\Khue\TDN\data\raw\ipp-sanluong-2024.xlsx"
# xls = pd.ExcelFile(file_path)
# sheet_names = xls.sheet_names

# # Xử lý mỗi sheet và kết hợp chúng thành 1 DataFrame
# dfs = []
# for sheet in sheet_names:
#     # Đọc sheet thành pandas DataFrame
#     df = pd.read_excel(file_path, sheet_name=sheet)
#     # Chuyển đổi thành Dask DataFrame
#     ddf = dd.from_pandas(df, npartitions=10)  # Chia thành 10 phần
#     dfs.append(ddf)

# # Ghép tất cả Dask DataFrame
# combined_df = dd.concat(dfs)

In [2]:
# import pandas as pd
# import os

# # Cấu hình đường dẫn
# raw_folder = r"C:\Khue\TDN\data\raw"
# out_folder = r"C:\Khue\TDN\data\processed"

# # Tên file Excel theo năm
# file_years = ['2021', '2022', '2023', '2024']

# # Đọc từng file và lưu thành parquet
# for year in file_years:
#     file_path = os.path.join(raw_folder, f"ipp-sanluong-{year}.xlsx")
#     print(f"Đang xử lý file {file_path}...")

#     # Đọc toàn bộ sheet
#     xls = pd.ExcelFile(file_path)
#     year_dfs = []

#     for sheet in xls.sheet_names:
#         df = pd.read_excel(file_path, sheet_name=sheet, engine='openpyxl')
#         df['NAM'] = int(year)
#         year_dfs.append(df)

#     # Gộp tất cả sheet lại
#     df_year = pd.concat(year_dfs, ignore_index=True)

#     # Lưu thành parquet
#     output_path = os.path.join(out_folder, f"sanluong_{year}.parquet")
#     df_year.to_parquet(output_path, index=False, compression='snappy')
#     print(f"Đã lưu file: {output_path}")


In [3]:
# # Chuyển đổi kết quả từ dask DataFrame sang pandas DataFrame
# result_df = combined_df.compute()
# # reset index
# result_df.reset_index(drop=True, inplace=True)
# result_df

In [17]:
import pandas as pd
import os

# Thư mục chứa các file parquet
parquet_folder = r"C:\Khue\TDN\data\processed"

# Tự động tìm tất cả các file parquet theo mẫu
parquet_files = [os.path.join(parquet_folder, f) 
                 for f in os.listdir(parquet_folder) 
                 if f.startswith("sanluong_") and f.endswith(".parquet")]

# Đọc và ghép toàn bộ file
df_all = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)

# Kiểm tra tổng số dòng sau khi ghép
print(f"✅ Đã đọc xong {len(parquet_files)} file parquet.")
print(f"👉 Tổng số dòng dữ liệu: {df_all.shape[0]:,}")


✅ Đã đọc xong 4 file parquet.
👉 Tổng số dòng dữ liệu: 18,404,864


In [18]:
df_all

CTDL             NMTD        MADIEMDO  \
0              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
1              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
2              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
3              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
4              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
...                            ...              ...             ...   
18404859  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404860  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404861  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404862  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404863  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   

                TENDIEMDO  SOCONGTO           STARTTIME             ENDTIME  \
0               Chính-331  40377597 2021-01-01 00:00:00 2021-01-01 00:30:00   
1               Chính-331  40377597 2021-01-01 00:30:00 2021-01-01 01:00:00   
2               Chính-331  40377597 2021-01-01 01:00:00 2021-01-01 01:30:00   
3               Chính-331  40377597 2021-01-01 01:30:00 2021-01-01 02:00:00   
4               Chính-331  40377597 2021-01-01 02:00:00 2021-01-01 02:30:00   
...                   ...       ...                 ...                 ...   
18404859  Sông Lô 8B_131c  40774734 2024-08-07 01:30:00 2024-08-07 01:30:00   
18404860  Sông Lô 8B_131c  40774734 2024-08-07 02:00:00 2024-08-07 02:00:00   
18404861  Sông Lô 8B_131c  40774734 2024-08-07 05:30:00 2024-08-07 05:30:00   
18404862  Sông Lô 8B_131c  40774734 2024-08-07 06:00:00 2024-08-07 06:00:00   
18404863  Sông Lô 8B_131c  40774734 2024-08-07 00:30:00 2024-08-07 00:30:00   

            CS  SL_PGIAOTONG   NAM  
0          0.0    45795204.6  2021  
1          0.0    45795204.6  2021  
2          0.0    45795204.6  2021  
3          0.0    45795204.6  2021  
4          0.0    45795204.6  2021  
...        ...           ...   ...  
18404859   0.0   241770163.0  2024  
18404860   0.0   241770163.0  2024  
18404861   0.0   241770163.0  2024  
18404862  17.0   241770973.1  2024  
18404863   0.0   241770163.0  2024  

[18404864 rows x 10 columns]

In [19]:
df_sanluong = df_all[['CTDL','NMTD','MADIEMDO','ENDTIME','CS']]
df_sanluong.rename(columns={'ENDTIME':'TIME'}, inplace=True)
df_sanluong

C:\Users\nltt03\AppData\Local\Temp\ipykernel_13468\1643857597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sanluong.rename(columns={'ENDTIME':'TIME'}, inplace=True)


CTDL             NMTD        MADIEMDO  \
0              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
1              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
2              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
3              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
4              CTY ĐIỆN LỰC SƠN LA  NMTĐ SUỐI SẬP 2  G2A000S000M331   
...                            ...              ...             ...   
18404859  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404860  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404861  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404862  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   
18404863  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  G2A214S000M131   

                        TIME    CS  
0        2021-01-01 00:30:00   0.0  
1        2021-01-01 01:00:00   0.0  
2        2021-01-01 01:30:00   0.0  
3        2021-01-01 02:00:00   0.0  
4        2021-01-01 02:30:00   0.0  
...                      ...   ...  
18404859 2024-08-07 01:30:00   0.0  
18404860 2024-08-07 02:00:00   0.0  
18404861 2024-08-07 05:30:00   0.0  
18404862 2024-08-07 06:00:00  17.0  
18404863 2024-08-07 00:30:00   0.0  

[18404864 rows x 5 columns]

In [7]:
# Group the DataFrame by the 'CTDL' column
grouped_by_ctdl = df_sanluong.groupby('CTDL')

# Display the number of entries in each group to confirm
print("Number of entries per CTDL group:")
print(grouped_by_ctdl.size())


Number of entries per CTDL group:
CTDL
CTY ĐIỆN LỰC BẮC KẠN         226656
CTY ĐIỆN LỰC CAO BẰNG        931890
CTY ĐIỆN LỰC HÀ GIANG       1809836
CTY ĐIỆN LỰC HÀ TĨNH         185878
CTY ĐIỆN LỰC HÒA BÌNH        294436
CTY ĐIỆN LỰC LAI CHÂU       1834029
CTY ĐIỆN LỰC LÀO CAI        2571109
CTY ĐIỆN LỰC LẠNG SƠN        256700
CTY ĐIỆN LỰC NGHỆ AN         903629
CTY ĐIỆN LỰC QUẢNG NINH       44432
CTY ĐIỆN LỰC SƠN LA         2609270
CTY ĐIỆN LỰC THANH HÓA       425065
CTY ĐIỆN LỰC THÁI NGUYÊN      43481
CTY ĐIỆN LỰC TUYÊN QUANG     178235
CTY ĐIỆN LỰC YÊN BÁI        1079626
CTY ĐIỆN LỰC ĐIỆN BIÊN       726411
dtype: int64


In [ ]:
# df_laocai = df_sanluong[df_sanluong['CTDL'] == 'CTY ĐIỆN LỰC SƠN LA'].copy()
# df_laocai.reset_index(drop=True, inplace=True)
# df_laocai

CTDL              NMTD        MADIEMDO  \
0       CTY ĐIỆN LỰC SƠN LA      NMTĐ NẬM PIA  G2A003S000M131   
1       CTY ĐIỆN LỰC SƠN LA      NMTĐ NẬM PIA  G2A003S000M131   
2       CTY ĐIỆN LỰC SƠN LA      NMTĐ NẬM PIA  G2A003S000M131   
3       CTY ĐIỆN LỰC SƠN LA      NMTĐ NẬM PIA  G2A003S000M131   
4       CTY ĐIỆN LỰC SƠN LA      NMTĐ NẬM PIA  G2A003S000M131   
...                     ...               ...             ...   
635944  CTY ĐIỆN LỰC SƠN LA  NMTĐ Nậm Công 3A  G2A268S000M371   
635945  CTY ĐIỆN LỰC SƠN LA  NMTĐ Nậm Công 3A  G2A268S000M371   
635946  CTY ĐIỆN LỰC SƠN LA  NMTĐ Nậm Công 3A  G2A268S000M371   
635947  CTY ĐIỆN LỰC SƠN LA  NMTĐ Nậm Công 3A  G2A268S000M371   
635948  CTY ĐIỆN LỰC SƠN LA  NMTĐ Nậm Công 3A  G2A268S000M371   

                      TIME       CS  
0      2024-08-07 20:30:00  14805.0  
1      2024-08-07 19:30:00  14809.0  
2      2024-08-08 06:00:00  14772.0  
3      2024-08-08 08:00:00  14778.0  
4      2024-08-08 08:30:00  14780.0  
...                    ...      ...  
635944 2024-01-02 18:00:00   1688.4  
635945 2024-01-02 11:00:00      0.0  
635946 2024-01-02 17:30:00      0.0  
635947 2024-01-02 16:00:00      0.0  
635948 2024-01-02 00:30:00      0.0  

[635949 rows x 5 columns]

In [ ]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Add month column to df_laocai if it doesn't exist
if 'MONTH' not in df_sanluong.columns:
    df_sanluong['MONTH'] = df_sanluong['TIME'].dt.strftime('%Y-%m')

# Danh sách nhà máy và tháng
nmtd_list = sorted(df_sanluong['NMTD'].dropna().unique())
month_list = sorted(df_sanluong['MONTH'].dropna().unique())

# Dropdown widgets
nmtd_dropdown = widgets.Dropdown(options=nmtd_list, description='Nhà máy:')
month_dropdown = widgets.Dropdown(options=month_list, description='Tháng:')

# Hàm vẽ biểu đồ tương tác
def plot_filtered(nmtd_value, month_value):
    filtered = df_sanluong[
        (df_sanluong['NMTD'] == nmtd_value) &
        (df_sanluong['MONTH'] == month_value)
    ]
    if filtered.empty:
        print("Không có dữ liệu.")
        return
    filtered = filtered.sort_values('TIME')
    fig = px.line(
        filtered,
        x='TIME', y='CS',
        color='MADIEMDO',
        title=f"Sản lượng theo thời gian - {nmtd_value} - {month_value}",
        markers=False
        
        
        
        
        
    )
    fig.update_layout(
        xaxis_title='Thời gian',
        yaxis_title='Sản lượng (CS)',
        legend_title='Mã điểm đo'
    )
    fig.show()

# Tương tác 2 dropdown
widgets.interact(plot_filtered, nmtd_value=nmtd_dropdown, month_value=month_dropdown)

# # Hiển thị tổng quan
# fig = px.line(df_sanluong, x='TIME', y='CS', color='MADIEMDO',
#               title='Sản lượng theo thời gian - Lào Cai')
# fig.show()

In [4]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

# Thêm cột YEAR và MONTH nếu chưa có
df_sanluong['YEAR'] = df_sanluong['TIME'].dt.year
df_sanluong['MONTH_NUM'] = df_sanluong['TIME'].dt.month

# Dropdown CTDL
ctdl_list = sorted(df_sanluong['CTDL'].dropna().unique())
ctdl_dropdown = widgets.Dropdown(options=ctdl_list, description='CTDL:')

# Dropdown NMTD (cập nhật theo CTDL)
nmtd_dropdown = widgets.Dropdown(description='Nhà máy:')

# Dropdown YEAR và MONTH
year_list = sorted(df_sanluong['YEAR'].dropna().unique())
year_dropdown = widgets.Dropdown(options=year_list, description='Năm:')
month_dropdown = widgets.Dropdown(options=list(range(1, 13)), description='Tháng:')

# Cập nhật nhà máy khi chọn CTDL
def update_nmtd(ctdl_value):
    filtered = df_sanluong[df_sanluong['CTDL'] == ctdl_value]
    nmtd_list = sorted(filtered['NMTD'].dropna().unique())
    nmtd_dropdown.options = nmtd_list

# Cập nhật NMTD khi người dùng chọn CTDL (KHÔNG render widget)
ctdl_dropdown.observe(lambda change: update_nmtd(change['new']), names='value')

# Khởi tạo giá trị ban đầu cho NMTD
update_nmtd(ctdl_dropdown.value)

# Hàm vẽ biểu đồ
def plot_filtered(ctdl_value, nmtd_value, year_value, month_value):
    filtered = df_sanluong[
        (df_sanluong['CTDL'] == ctdl_value) &
        (df_sanluong['NMTD'] == nmtd_value) &
        (df_sanluong['YEAR'] == year_value) &
        (df_sanluong['MONTH_NUM'] == month_value)
    ]
    if filtered.empty:
        print("Không có dữ liệu.")
        return

    filtered = filtered.sort_values('TIME')

    fig = px.line(
        filtered,
        x='TIME',
        y='CS',
        color='MADIEMDO',
        title=f"Sản lượng theo thời gian - {nmtd_value} - {month_value}/{year_value}",
        markers=False
    )

    # Cập nhật layout và font phù hợp cho tiếng Việt, màu chữ đen
    fig.update_layout(
        xaxis_title='Thời gian',  # Trục x
        yaxis_title='Giá trị công suất',  # Trục y - Đổi tên
        legend_title='Mã điểm đo',
        height=600,
        hovermode='x unified',
        title={
            'text': f"Sản lượng theo thời gian - {nmtd_value} - {month_value}/{year_value}",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'size': 18, 'family': 'Arial Unicode MS', 'weight': 'bold', 'color': 'black'}  # Màu chữ đen cho tiêu đề
        },
        xaxis=dict(
            title_font=dict(size=14, family='Arial Unicode MS', weight='bold', color='black'),  # In đậm trục x, màu chữ đen
            tickfont=dict(size=12, family='Arial Unicode MS', color='black')  # Màu chữ đen trục x
        ),
        yaxis=dict(
            title_font=dict(size=14, family='Arial Unicode MS', weight='bold', color='black'),  # In đậm trục y, màu chữ đen
            tickfont=dict(size=12, family='Arial Unicode MS', color='black')  # Màu chữ đen trục y
        ),
        margin=dict(l=60, r=40, t=60, b=60)
    )
    fig.show()

# Tạo giao diện tương tác 4 filter
widgets.interact(
    plot_filtered,
    ctdl_value=ctdl_dropdown,
    nmtd_value=nmtd_dropdown,
    year_value=year_dropdown,
    month_value=month_dropdown
)


interactive(children=(Dropdown(description='CTDL:', options=('CTY ĐIỆN LỰC BẮC KẠN', 'CTY ĐIỆN LỰC CAO BẰNG', …

<function __main__.plot_filtered(ctdl_value, nmtd_value, year_value, month_value)>

In [20]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import calendar

# ========== TẠO CỘT THỜI GIAN ==========
df_sanluong['YEAR'] = df_sanluong['TIME'].dt.year
df_sanluong['MONTH_NUM'] = df_sanluong['TIME'].dt.month
df_sanluong['DAY_NUM'] = df_sanluong['TIME'].dt.day

# ========== DANH SÁCH ==========
ctdl_list = sorted(df_sanluong['CTDL'].dropna().unique())
year_list = sorted(df_sanluong['YEAR'].dropna().unique())

# ========== HÀM TẠO DROPDOWN ==========
def make_dropdown(options, description, width='250px', margin='0px 20px 0px 0px'):
    return widgets.Dropdown(
        options=options,
        description=description,
        layout=widgets.Layout(width=width, margin=margin),
        style={'description_width': 'auto'}
    )

# ========== DROPDOWNS ==========
ctdl_dropdown_month = make_dropdown(ctdl_list, 'CTDL:', '300px')
nmtd_dropdown_month = make_dropdown([], 'Nhà máy:', '300px')
year_dropdown_month = make_dropdown(year_list, 'Năm:', '120px')
month_dropdown_month = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
day_placeholder = widgets.Label(value='', layout=widgets.Layout(width='120px', margin='0px 20px 0px 0px'))

ctdl_dropdown_day = make_dropdown(ctdl_list, 'CTDL:', '300px')
nmtd_dropdown_day = make_dropdown([], 'Nhà máy:', '300px')
year_dropdown_day = make_dropdown(year_list, 'Năm:', '120px')
month_dropdown_day = make_dropdown(list(range(1, 13)), 'Tháng:', '120px')
day_dropdown_day = make_dropdown(list(range(1, 32)), 'Ngày:', '120px')

# ========== NHÓM WIDGET ==========
month_widgets = [ctdl_dropdown_month, nmtd_dropdown_month, year_dropdown_month, month_dropdown_month]
day_widgets = [ctdl_dropdown_day, nmtd_dropdown_day, year_dropdown_day, month_dropdown_day, day_dropdown_day]

# ========== CẬP NHẬT NHÀ MÁY ==========
def update_nmtd(ctdl_value, dropdown):
    filtered = df_sanluong[df_sanluong['CTDL'] == ctdl_value]
    options = sorted(filtered['NMTD'].dropna().unique())
    dropdown.options = options

ctdl_dropdown_month.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_month), names='value')
ctdl_dropdown_day.observe(lambda change: update_nmtd(change['new'], nmtd_dropdown_day), names='value')

update_nmtd(ctdl_dropdown_month.value, nmtd_dropdown_month)
update_nmtd(ctdl_dropdown_day.value, nmtd_dropdown_day)

# ========== GIỚI HẠN NGÀY ==========
def update_day_options(*args):
    year = year_dropdown_day.value
    month = month_dropdown_day.value
    if year and month:
        max_day = calendar.monthrange(year, month)[1]
        day_dropdown_day.options = list(range(1, max_day + 1))

year_dropdown_day.observe(update_day_options, names='value')
month_dropdown_day.observe(update_day_options, names='value')
update_day_options()

# ========== VẼ BIỂU ĐỒ ==========
def plot_filtered(mode, ctdl, nmtd, year, month, day=None):
    if mode == 'month':
        filtered = df_sanluong[
            (df_sanluong['CTDL'] == ctdl) &
            (df_sanluong['NMTD'] == nmtd) &
            (df_sanluong['YEAR'] == year) &
            (df_sanluong['MONTH_NUM'] == month)
        ]
        title_ext = f"{month}/{year}"
    else:
        filtered = df_sanluong[
            (df_sanluong['CTDL'] == ctdl) &
            (df_sanluong['NMTD'] == nmtd) &
            (df_sanluong['YEAR'] == year) &
            (df_sanluong['MONTH_NUM'] == month) &
            (df_sanluong['DAY_NUM'] == day)
        ]
        title_ext = f"{day}/{month}/{year}"

    if filtered.empty:
        with out:
            out.clear_output()
            print("Không có dữ liệu cho lựa chọn này.")
        return

    filtered = filtered.sort_values('TIME')

    fig = px.line(
        filtered,
        x='TIME',
        y='CS',
        color='MADIEMDO',
        render_mode='webgl',
        markers=True if mode == 'day' else False
    )

    fig.update_layout(
        title=None,
        annotations=[
            dict(
                text=f"<b>Công suất theo chu kỳ 30' - {nmtd} - {title_ext}</b>",
                xref='paper', yref='paper',
                x=0.45, y=1.1,
                xanchor='center', yanchor='top',
                showarrow=False,
                font=dict(size=18, color='black')
            )
        ],
        template='plotly_white',
        height=600,
        hovermode='x unified',

        xaxis_title=None,  # ❌ Ẩn tiêu đề mặc định trục X
        yaxis_title='Giá trị công suất',

        xaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            griddash='dot',
            title_font=dict(size=14, color='black', weight='bold'),
            ticklabelposition='outside',
            ticks='outside',
            ticklen=8
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgrey',
            griddash='dot',
            rangemode='tozero',
            title_font=dict(size=14, color='black', weight='bold'),
            ticklabelposition='outside',
            ticks='outside',
            ticklen=8
        ),

        margin=dict(l=60, r=40, t=80, b=80),
        paper_bgcolor='white',
        plot_bgcolor='#f0f8ff'
    )

    # ✅ Tên trục X bằng annotation
    fig.add_annotation(
        text="<b>Thời gian</b>",
        xref='paper', yref='paper',
        x=0.45, y=-0.15,
        showarrow=False,
        font=dict(size=14, color='black')
    )

    fig.update_traces(
        line=dict(width=2),
        hovertemplate='Thời gian: %{x}<br>Công suất: %{y}'
    )

    with out:
        out.clear_output()
        fig.show()

# ========== SỰ KIỆN ==========
def on_change_month(change):
    plot_filtered(
        'month',
        ctdl_dropdown_month.value,
        nmtd_dropdown_month.value,
        year_dropdown_month.value,
        month_dropdown_month.value
    )

def on_change_day(change):
    plot_filtered(
        'day',
        ctdl_dropdown_day.value,
        nmtd_dropdown_day.value,
        year_dropdown_day.value,
        month_dropdown_day.value,
        day_dropdown_day.value
    )

# ========== TẮT / BẬT OBSERVER ==========
def disable_observe(widgets, callback):
    for w in widgets:
        w.unobserve(callback, names='value')

def enable_observe(widgets, callback):
    for w in widgets:
        w.observe(callback, names='value')

# ========== GẮN OBSERVER ==========
for w in month_widgets:
    w.observe(on_change_month, 'value')

for w in day_widgets:
    w.observe(on_change_day, 'value')

# ========== GIAO DIỆN ==========
tab_thang = widgets.HBox([
    ctdl_dropdown_month,
    nmtd_dropdown_month,
    year_dropdown_month,
    month_dropdown_month,
    day_placeholder
], layout=widgets.Layout(justify_content='flex-start', overflow='visible'))

tab_ngay = widgets.HBox([
    ctdl_dropdown_day,
    nmtd_dropdown_day,
    year_dropdown_day,
    month_dropdown_day,
    day_dropdown_day
], layout=widgets.Layout(justify_content='flex-start', overflow='visible'))

tabs = widgets.Tab(children=[tab_thang, tab_ngay])
tabs.set_title(0, 'Xem theo tháng')
tabs.set_title(1, 'Xem theo ngày')

# ========== CHUYỂN TAB ==========
def on_tab_change(change):
    if change['name'] == 'selected_index':
        if change['new'] == 1:
            disable_observe(day_widgets, on_change_day)
            ctdl_dropdown_day.value = ctdl_dropdown_month.value
            nmtd_dropdown_day.value = nmtd_dropdown_month.value
            year_dropdown_day.value = year_dropdown_month.value
            month_dropdown_day.value = month_dropdown_month.value
            day_dropdown_day.value = 1
            enable_observe(day_widgets, on_change_day)
            on_change_day(None)
        elif change['new'] == 0:
            disable_observe(month_widgets, on_change_month)
            ctdl_dropdown_month.value = ctdl_dropdown_day.value
            nmtd_dropdown_month.value = nmtd_dropdown_day.value
            year_dropdown_month.value = year_dropdown_day.value
            month_dropdown_month.value = month_dropdown_day.value
            enable_observe(month_widgets, on_change_month)
            on_change_month(None)

tabs.observe(on_tab_change)

# ========== HIỂN THỊ ==========
out = widgets.Output()
display(tabs, out)


C:\Users\nltt03\AppData\Local\Temp\ipykernel_13468\1451559255.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sanluong['YEAR'] = df_sanluong['TIME'].dt.year
C:\Users\nltt03\AppData\Local\Temp\ipykernel_13468\1451559255.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sanluong['MONTH_NUM'] = df_sanluong['TIME'].dt.month
C:\Users\nltt03\AppData\Local\Temp\ipykernel_13468\1451559255.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

Output()

In [ ]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Thêm cột tháng nếu chưa có
if 'MONTH' not in df_laocai.columns:
    df_laocai['MONTH'] = df_laocai['TIME'].dt.strftime('%Y-%m')

# Dropdown widgets cơ bản
nmtd_list = sorted(df_laocai['NMTD'].dropna().unique())
month_list = sorted(df_laocai['MONTH'].dropna().unique())

nmtd_dropdown = widgets.Dropdown(options=nmtd_list, description='Nhà máy:')
month_dropdown = widgets.Dropdown(options=month_list, description='Tháng:')
day_dropdown = widgets.Dropdown(description='Ngày:')

# Hàm cập nhật ngày theo tháng và nhà máy
def update_days(nmtd_value, month_value):
    filtered_month = df_laocai[
        (df_laocai['NMTD'] == nmtd_value) &
        (df_laocai['MONTH'] == month_value)
    ]
    # Lấy danh sách ngày duy nhất có dữ liệu
    days = sorted(filtered_month['TIME'].dt.day.dropna().unique())
    day_dropdown.options = days

# Gọi khi chọn lại tháng/nhà máy
widgets.interactive(update_days, nmtd_value=nmtd_dropdown, month_value=month_dropdown)

# Hàm vẽ biểu đồ
def plot_filtered(nmtd_value, month_value, day_value):
    filtered = df_laocai[
        (df_laocai['NMTD'] == nmtd_value) &
        (df_laocai['MONTH'] == month_value) &
        (df_laocai['TIME'].dt.day == day_value)
    ]
    if filtered.empty:
        print("Không có dữ liệu.")
        return

    filtered = filtered.sort_values('TIME')
    fig = px.line(
        filtered,
        x='TIME', y='CS',
        color='MADIEMDO',
        title=f"Sản lượng theo thời gian - {nmtd_value} - {month_value} - Ngày {day_value}",
        markers=True
    )
    fig.update_layout(
        xaxis_title='Thời gian',
        yaxis_title='Sản lượng (CS)',
        legend_title='Mã điểm đo',
        width=1300,
        height=500
    )
    fig.show()

# Kết hợp 3 dropdown tương tác
widgets.interact(plot_filtered,
                 nmtd_value=nmtd_dropdown,
                 month_value=month_dropdown,
                 day_value=day_dropdown)


interactive(children=(Dropdown(description='Nhà máy:', options=('NMTĐ CHIỀNG CÔNG 2', 'NMTĐ CHIỀNG NGÀM THƯỢNG…

<function __main__.plot_filtered(nmtd_value, month_value, day_value)>

In [5]:
# chỉ giữ lại các cột MADIEMDO, STARTTIME, CS của result_df
df = result_df[["MADIEMDO", "STARTTIME", "CS"]]
df

MADIEMDO           STARTTIME      CS
0        G2A121S000M371 2024-08-07 23:30:00  3395.8
1        G2A121S000M371 2024-08-07 18:30:00  2984.8
2        G2A121S000M371 2024-08-07 19:00:00  2363.7
3        G2A121S000M371 2024-08-08 07:30:00  3325.8
4        G2A121S000M371 2024-08-08 08:00:00  3294.5
...                 ...                 ...     ...
3577158  G2A214S000M131 2024-08-07 01:30:00     0.0
3577159  G2A214S000M131 2024-08-07 02:00:00     0.0
3577160  G2A214S000M131 2024-08-07 05:30:00     0.0
3577161  G2A214S000M131 2024-08-07 06:00:00    17.0
3577162  G2A214S000M131 2024-08-07 00:30:00     0.0

[3577163 rows x 3 columns]

In [6]:
df["NGAY"] = df["STARTTIME"].dt.date
df["CHU_KY"] = df["STARTTIME"].dt.time
df = df.drop(columns=["STARTTIME"])
df

MADIEMDO      CS        NGAY    CHU_KY
0        G2A121S000M371  3395.8  2024-08-07  23:30:00
1        G2A121S000M371  2984.8  2024-08-07  18:30:00
2        G2A121S000M371  2363.7  2024-08-07  19:00:00
3        G2A121S000M371  3325.8  2024-08-08  07:30:00
4        G2A121S000M371  3294.5  2024-08-08  08:00:00
...                 ...     ...         ...       ...
3577158  G2A214S000M131     0.0  2024-08-07  01:30:00
3577159  G2A214S000M131     0.0  2024-08-07  02:00:00
3577160  G2A214S000M131     0.0  2024-08-07  05:30:00
3577161  G2A214S000M131    17.0  2024-08-07  06:00:00
3577162  G2A214S000M131     0.0  2024-08-07  00:30:00

[3577163 rows x 4 columns]

In [7]:
# Convert CHU_KY to datetime format first
df["CHU_KY"] = pd.to_datetime(df["CHU_KY"].astype(str)).dt.time

# Filter rows where minutes are multiples of 30 and second is always = 00
df = df[
    (pd.to_datetime(df["CHU_KY"].astype(str)).dt.minute % 30 == 0)
    & (pd.to_datetime(df["CHU_KY"].astype(str)).dt.second == 0)
]
df


MADIEMDO      CS        NGAY    CHU_KY
0        G2A121S000M371  3395.8  2024-08-07  23:30:00
1        G2A121S000M371  2984.8  2024-08-07  18:30:00
2        G2A121S000M371  2363.7  2024-08-07  19:00:00
3        G2A121S000M371  3325.8  2024-08-08  07:30:00
4        G2A121S000M371  3294.5  2024-08-08  08:00:00
...                 ...     ...         ...       ...
3577158  G2A214S000M131     0.0  2024-08-07  01:30:00
3577159  G2A214S000M131     0.0  2024-08-07  02:00:00
3577160  G2A214S000M131     0.0  2024-08-07  05:30:00
3577161  G2A214S000M131    17.0  2024-08-07  06:00:00
3577162  G2A214S000M131     0.0  2024-08-07  00:30:00

[3573513 rows x 4 columns]

In [ ]:
# Chuyển file sản lượng thành dạng chia theo ngày và 48 chu kỳ
new_data = pd.pivot_table(
    df, values="CS", index=["MADIEMDO", "NGAY"], columns="CHU_KY"
).reset_index()
new_data


CHU_KY        MADIEMDO        NGAY  00:00:00  00:30:00  01:00:00  01:30:00  \
0             15098879  2024-05-12     0.000     0.000     0.000      0.00   
1             15098879  2024-05-13     0.000     0.000     0.000      0.00   
2             15098879  2024-05-14     0.000     0.000     0.000      0.00   
3             15098879  2024-05-15     0.000     0.000     0.000      0.00   
4             15098879  2024-05-16     0.000     0.000     0.000      0.00   
...                ...         ...       ...       ...       ...       ...   
74987   G2A298S000M131  2024-08-02     0.084     0.011     0.000      0.00   
74988   G2A298S000M131  2024-08-03     0.000     0.000     0.000      0.00   
74989   G2A298S000M131  2024-08-04     0.000     0.000     0.000      0.00   
74990   G2A298S000M131  2024-08-05     0.000     0.000     0.000      0.00   
74991   G2A298S000M131  2024-08-06     0.275     0.290     0.291      0.29   

CHU_KY  02:00:00  02:30:00  03:00:00  03:30:00  ...  19:00:00  19:30:00  \
0          0.000      0.00     0.000     0.000  ...     0.000     0.000   
1          0.000      0.00     0.000     0.000  ...  1401.100   793.260   
2          0.000      0.00     0.000     0.000  ...  1660.100  1062.300   
3          0.000      0.00     0.000     0.000  ...  1229.200   853.500   
4          0.000      0.00     0.000     0.000  ...  1971.800  1951.900   
...          ...       ...       ...       ...  ...       ...       ...   
74987      0.000      0.00     0.000     0.000  ...     0.000     0.116   
74988      0.000      0.00     0.000     0.000  ...     0.168     0.185   
74989      0.000      0.00     0.000     0.000  ...     0.000     0.026   
74990      0.000      0.00     0.000     0.000  ...     0.287     0.240   
74991      0.291      0.29     0.291     0.291  ...     0.291     0.290   

CHU_KY  20:00:00  20:30:00  21:00:00  21:30:00  22:00:00  22:30:00  23:00:00  \
0          0.000     0.000     0.000     0.000     0.000     0.000     0.000   
1          0.000     0.000     0.000     0.000     0.000     0.000     0.000   
2          0.080     0.000     0.000     0.000     0.000     0.000     0.000   
3          0.000     0.000     0.000     0.000     0.000     0.000     0.000   
4       1733.200  1593.700  1564.400  1362.500  1319.600  1282.100  1062.100   
...          ...       ...       ...       ...       ...       ...       ...   
74987      0.123     0.109     0.077     0.058     0.000     0.006     0.072   
74988      0.159     0.081     0.000     0.000     0.000     0.000     0.000   
74989      0.000     0.025     0.151     0.028     0.097     0.140     0.147   
74990      0.000     0.000     0.067     0.270     0.024     0.107     0.000   
74991      0.290     0.291       NaN       NaN       NaN       NaN       NaN   

CHU_KY  23:30:00  
0          0.000  
1          0.000  
2          0.000  
3          0.000  
4         27.160  
...          ...  
74987      0.000  
74988      0.000  
74989      0.000  
74990      0.005  
74991        NaN  

[74992 rows x 50 columns]

In [47]:
# Các chị để đường dẫn file thông tin NMTD
file_path_2 = r"C:\Khue\TDN\data\raw\thong_tin_NMTD_convert.xlsx"
df_NMTD = pd.read_excel(file_path_2)
df_NMTD

DVDL             NMTD      TEN_NM        MADIEMDO
0        CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000D371
1        CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000M371
2        CTY ĐIỆN LỰC BẮC KẠN   NMTĐ THƯỢNG ÂN   THUONG_AN  G2A122S000D371
3        CTY ĐIỆN LỰC BẮC KẠN   NMTĐ THƯỢNG ÂN   THUONG_AN  G2A122S000M371
4        CTY ĐIỆN LỰC BẮC KẠN     NMTĐ NẬM CẮT     NAM_CAT  G2A123S000M371
..                        ...              ...         ...             ...
802  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8A  SONG_LO_8A  G2A203S000D131
803  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000D132
804  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M132
805  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000D131
806  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131

[807 rows x 4 columns]

In [ ]:
# nCác chị để đường dẫn file DS_TTT_TDN
df_TDN = pd.read_excel(r"C:\Khue\TDN\data\raw\DS_TTT_TDN.xlsx")

In [ ]:
# Merge TD_THAMCHIEU cho file thông tin NMTD bằng cột key là 'TEN_NM'
merged_df = df_NMTD.merge(df_TDN[["TEN_NM", "TD_THAMCHIEU"]], on="TEN_NM", how="left")
merged_df

DVDL             NMTD      TEN_NM        MADIEMDO  \
0        CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000D371   
1        CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000M371   
2        CTY ĐIỆN LỰC BẮC KẠN   NMTĐ THƯỢNG ÂN   THUONG_AN  G2A122S000D371   
3        CTY ĐIỆN LỰC BẮC KẠN   NMTĐ THƯỢNG ÂN   THUONG_AN  G2A122S000M371   
4        CTY ĐIỆN LỰC BẮC KẠN     NMTĐ NẬM CẮT     NAM_CAT  G2A123S000M371   
..                        ...              ...         ...             ...   
804  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8A  SONG_LO_8A  G2A203S000D131   
805  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000D132   
806  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M132   
807  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000D131   
808  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131   

    TD_THAMCHIEU  
0    TUYEN_QUANG  
1    TUYEN_QUANG  
2    TUYEN_QUANG  
3    TUYEN_QUANG  
4    TUYEN_QUANG  
..           ...  
804  TUYEN_QUANG  
805  TUYEN_QUANG  
806  TUYEN_QUANG  
807  TUYEN_QUANG  
808  TUYEN_QUANG  

[809 rows x 5 columns]

In [ ]:
# Merge sản lượng từ file sản lượng ipp vào File thông tin NMTD với cột key là 'MADIEMDO'
merged_df_1 = merged_df.merge(new_data, on="MADIEMDO", how="left")
merged_df_1

DVDL             NMTD      TEN_NM        MADIEMDO  \
0          CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000D371   
1          CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000M371   
2          CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000M371   
3          CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000M371   
4          CTY ĐIỆN LỰC BẮC KẠN     NMTĐ TÀ LÀNG     TA_LANG  G2A121S000M371   
...                         ...              ...         ...             ...   
77883  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131   
77884  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131   
77885  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131   
77886  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131   
77887  CTY ĐIỆN LỰC TUYÊN QUANG  NMTĐ Sông Lô 8B  SONG_LO_8B  G2A214S000M131   

      TD_THAMCHIEU        NGAY  00:00:00  00:30:00  01:00:00  01:30:00  ...  \
0      TUYEN_QUANG         NaN       NaN       NaN       NaN       NaN  ...   
1      TUYEN_QUANG  2024-01-01       0.0       0.0       0.0       0.0  ...   
2      TUYEN_QUANG  2024-01-02       0.0       0.0       0.0       0.0  ...   
3      TUYEN_QUANG  2024-01-03       0.0       0.0       0.0       0.0  ...   
4      TUYEN_QUANG  2024-01-04       0.0       0.0       0.0       0.0  ...   
...            ...         ...       ...       ...       ...       ...  ...   
77883  TUYEN_QUANG  2024-08-04      64.0      64.0      64.0      63.0  ...   
77884  TUYEN_QUANG  2024-08-05       0.0       0.0       0.0       0.0  ...   
77885  TUYEN_QUANG  2024-08-06       0.0       0.0       0.0       0.0  ...   
77886  TUYEN_QUANG  2024-08-07       0.0       0.0       0.0       0.0  ...   
77887  TUYEN_QUANG  2024-08-08      36.0      36.0      35.0      35.0  ...   

       19:00:00  19:30:00  20:00:00  20:30:00  21:00:00  21:30:00  22:00:00  \
0           NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1        2394.0    831.68    332.08       0.0       0.0       0.0       0.0   
2        2058.0    754.40    343.84       0.0       0.0       0.0       0.0   
3        1879.9    712.94      1.12       0.0       0.0       0.0       0.0   
4        2076.5   1163.10    427.34       0.0       0.0       0.0       0.0   
...         ...       ...       ...       ...       ...       ...       ...   
77883       0.0      0.00      0.00       0.0       0.0       0.0       0.0   
77884       0.0      0.00      0.00       0.0       0.0       0.0       0.0   
77885       0.0      0.00      0.00       0.0       0.0       0.0       0.0   
77886      17.0     42.00     40.00      39.0      38.0      38.0      37.0   
77887       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

       22:30:00  23:00:00  23:30:00  
0           NaN       NaN       NaN  
1           0.0       0.0       0.0  
2           0.0       0.0       0.0  
3           0.0       0.0       0.0  
4           0.0       0.0       0.0  
...         ...       ...       ...  
77883       0.0       0.0       0.0  
77884       0.0       0.0       0.0  
77885       0.0       0.0       0.0  
77886      35.0      36.0      35.0  
77887       NaN       NaN       NaN  

[77888 rows x 54 columns]

In [ ]:
# xoá những dòng nào mà tất cả giá trị từ 00:00:00 đến 23:30:00 đều blank
merged_df_2 = merged_df_1.dropna(subset=merged_df_1.columns[6:], how="all")

In [ ]:
# Export ra file excel
merged_df_2.to_excel(r"C:\Khue\TDN\data\processed\2024.xlsx", index=False)

In [10]:
import os
import pandas as pd

# Đường dẫn thư mục chứa file
folder_path = r"C:\Khue\TDN\data\processed"

# Danh sách tên file bạn muốn đọc (chỉ 4 file của các năm)
file_names = ["2021.xlsx", "2022.xlsx", "2023.xlsx", "2024.xlsx"]

# Tạo đường dẫn đầy đủ cho từng file
files = [os.path.join(folder_path, file) for file in file_names]

# Đọc từng file vào DataFrame
dfs = [pd.read_excel(file) for file in files]

# Gộp các DataFrame lại với nhau theo chiều dọc
merged_df = pd.concat(dfs, ignore_index=True)

# Lưu DataFrame đã gộp ra file Excel mới trong cùng thư mục
output_file = os.path.join(folder_path, "merged_data.xlsx")
merged_df.to_excel(output_file, index=False)

print(f"Đã gộp {len(files)} file thành công vào file: {output_file}")


Đã gộp 4 file thành công vào file: C:\Khue\TDN\data\processed\merged_data.xlsx


In [13]:
# Chuyển cột thời gian 'START_TIME' sang kiểu datetime để sắp xếp chính xác
merged_df['NGAY'] = pd.to_datetime(merged_df['NGAY'], errors='coerce')

# Sắp xếp dữ liệu trong từng nhóm 'MADIEMDO' theo 'START_TIME'
df_sorted = merged_df.groupby('MADIEMDO', group_keys=False).apply(lambda x: x.sort_values(by='NGAY'))

# Lưu lại dữ liệu đã sắp xếp vào file mới
output_file = "sanluongipp_sorted.xlsx"
df_sorted.to_excel(output_file, index=False)

print(f"Đã sắp xếp dữ liệu theo 'MADIEMDO' và 'START_TIME', lưu vào: {output_file}")


Đã sắp xếp dữ liệu theo 'MADIEMDO' và 'START_TIME', lưu vào: sanluongipp_sorted.xlsx
